# Multiple choice on IPU using RoBERTa - Fine-tuning

In this notebook, we will see how to fine-tune one of the [🤗 Transformers](https://github.com/huggingface/transformers) models to a multiple choice task, which is the task of selecting the most plausible inputs in a given selection. We will use [SWAG](https://www.aclweb.org/anthology/D18-1009/) dataset but you can adapt the pre-processing parts of this notebook to any other multiple choice dataset you like. You can also use your own data. SWAG is a dataset about common-sense reasoning, where each sample describes a situation then proposes four options that could go after it. 

|  Domain | Tasks | Model | Datasets | Workflow |   Number of IPUs   | Execution time |
|---------|-------|-------|----------|----------|--------------|--------------|
| Natural language processing | Text classification | roberta-base | SWAG | Fine-tuning | 4 | 16min |

[![Join our Slack Community](https://img.shields.io/badge/Slack-Join%20Graphcore's%20Community-blue?style=flat-square&logo=slack)](https://www.graphcore.ai/join-community)

## Environment setup

The best way to run this demo is on Paperspace Gradient's cloud IPUs because everything is already set up for you.

[![Run on Gradient](https://assets.paperspace.io/img/gradient-badge.svg)](https://ipu.dev/3lI3fzQ)

To run the demo using other IPU hardware, you need to have the Poplar SDK enabled. Refer to the [Getting Started guide](https://docs.graphcore.ai/en/latest/getting-started.html#getting-started) for your system for details on how to enable the Poplar SDK. Also refer to the [Jupyter Quick Start guide](https://docs.graphcore.ai/projects/jupyter-notebook-quick-start/en/latest/index.html) for how to set up Jupyter to be able to run this notebook on a remote IPU machine.

## Dependencies and configuration

In order to improve usability and support for future users, Graphcore would like to collect information about the
applications and code being run in this notebook. The following information will be anonymised before being sent to Graphcore:

- User progression through the notebook
- Notebook details: number of cells, code being run and the output of the cells
- Environment details

You can disable logging at any time by running `%unload_ext graphcore_cloud_tools.notebook_logging.gc_logger` from any cell.

Install the dependencies for this notebook.

In [ ]:
%pip install "optimum-graphcore==0.7"
%pip install graphcore-cloud-tools[logger]@git+https://github.com/graphcore/graphcore-cloud-tools
%load_ext graphcore_cloud_tools.notebook_logging.gc_logger


This notebook is built to run with any model checkpoint from the [🤗 Model Hub](https://huggingface.co/models) as long as that model has a version with a multiple choice head and is supported by Optimum Graphcore. The IPU config files of the supported models are available in Graphcore's [🤗 account](https://huggingface.co/Graphcore). You can also create your own IPU config file locally. 

In this notebook, we are using both data parallelism and pipeline parallelism (see the [tutorial on efficient data loading](https://github.com/graphcore/examples/tree/master/tutorials/pytorch/efficient_data_loading) for more information). Therefore the global batch size, which is the actual number of samples used for the weight update, is determined with three factors:
- `global_batch_size = micro_batch_size * gradient_accumulation_steps * replication_factor`

The replication factor is determined by the type of IPU Pod, which will be used as a key to select the replication factor from a dictionary defined in the IPU config file. For example, the dictionary in the IPU config file [Graphcore/gpt2-small-ipu](https://huggingface.co/Graphcore/gpt2-small-ipu/blob/main/ipu_config.json) looks like this:
- `"replication_factor": {"pod4": 1, "pod8": 2, "pod16": 4, "pod32": 8, "pod64": 16, "default": 1}`

Depending on your model and the IPU Pod you are using, you might need to adjust these three batch-size-related arguments.

In [ ]:
model_checkpoint = "roberta-base"

ipu_config_name = "Graphcore/roberta-base-ipu"
micro_batch_size = 2
gradient_accumulation_steps = 16

Values for machine size and cache directories can be configured through environment variables or directly in the notebook:

In [ ]:
import os

n_ipu = int(os.getenv("NUM_AVAILABLE_IPU", 4))
executable_cache_dir = os.getenv("POPLAR_EXECUTABLE_CACHE_DIR", "/tmp/exe_cache/") + "/multiple_choice"

### Sharing your model with the community 

You can share your model with the 🤗 community. You do this by completing the following steps:

1. Store your authentication token from the 🤗 website. [Sign up to 🤗](https://huggingface.co/join) if you haven't already.
2. Execute the following cell and input your username and password.

In [ ]:
from huggingface_hub import notebook_login

notebook_login()

Then you need to install Git-LFS to manage large files:

In [ ]:
!apt install git-lfs

## Loading the dataset

We will use the [🤗 Datasets](https://github.com/huggingface/datasets) library to download the data. This can be easily done with the function `load_dataset`.  

In [ ]:
from datasets import load_dataset, load_metric

`load_dataset` will cache the dataset to avoid downloading it again the next time you run this cell.

In [ ]:
datasets = load_dataset("swag", "regular")

The `dataset` object itself is [`DatasetDict`](https://huggingface.co/docs/datasets/package_reference/main_classes.html#datasetdict), which contains one key for the training, validation and test set (with more keys for the mismatched validation and test sets in the special case of `mnli`).

In [ ]:
datasets

To access an actual element, you need to select a split ("train" in the example), and then specify an index:

In [ ]:
datasets["train"][0]

To get a sense of what the data looks like, the following function will show some samples picked randomly from the dataset.

In [ ]:
from datasets import ClassLabel
import random
import pandas as pd
from IPython.display import display, HTML

def show_random_elements(dataset, num_examples=10):
    assert num_examples <= len(dataset), "Can't pick more elements than there are in the dataset."
    picks = []
    for _ in range(num_examples):
        pick = random.randint(0, len(dataset)-1)
        while pick in picks:
            pick = random.randint(0, len(dataset)-1)
        picks.append(pick)
    
    df = pd.DataFrame(dataset[picks])
    for column, typ in dataset.features.items():
        if isinstance(typ, ClassLabel):
            df[column] = df[column].transform(lambda i: typ.names[i])
    display(HTML(df.to_html()))

In [ ]:
show_random_elements(datasets["train"])

Each sample in the dataset has a context composed of a first sentence (in the field `sent1`) and an introduction to the second sentence (in the field `sent2`). Then four possible endings are given (in the fields `ending0`, `ending1`, `ending2` and `ending3`) and the model must pick the right one (indicated in the field `label`). The following function lets us visualize a given sample:

In [ ]:
def show_one(example):
    print(f"Context: {example['sent1']}")
    print(f"  A - {example['sent2']} {example['ending0']}")
    print(f"  B - {example['sent2']} {example['ending1']}")
    print(f"  C - {example['sent2']} {example['ending2']}")
    print(f"  D - {example['sent2']} {example['ending3']}")
    print(f"\nGround truth: option {['A', 'B', 'C', 'D'][example['label']]}")

In [ ]:
show_one(datasets["train"][0])

In [ ]:
show_one(datasets["train"][15])

## Preprocessing the data

Before we can feed those text samples to our model, we need to preprocess them. This is done by a 🤗 Transformers `Tokenizer` which will tokenize the inputs (including converting the tokens to their corresponding IDs in the pre-trained vocabulary) and put it in a format the model expects, as well as generate the other inputs that the model requires.

To do all of this, we instantiate our tokenizer with the `AutoTokenizer.from_pretrained` method, which will ensure:

- We get a tokenizer that corresponds to the model architecture we want to use.
- We download the vocabulary used when pre-training this specific checkpoint.

That vocabulary will be cached, so it's not downloaded again the next time we run the cell.

In [ ]:
from transformers import AutoTokenizer
    
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint, use_fast=True)

We pass `use_fast=True` to the call above to use one of the fast tokenizers (backed by Rust) from the 🤗 Tokenizers library. These fast tokenizers are available for almost all models, but if you got an error with the previous call, remove that argument.

You can directly call this tokenizer on one sentence or a pair of sentences:

In [ ]:
tokenizer("Hello, this one sentence!", "And this sentence goes with it.")

Depending on the model you selected, you will see different keys in the dictionary returned by the cell above. They don't matter much for what we're doing here, but they are required by the model we will instantiate later. You can learn more about them in this [tutorial on preprocessing](https://huggingface.co/transformers/preprocessing.html).

To preprocess our dataset, we will thus need the names of the columns containing the sentence(s). The following dictionary keeps track of the correspondence of task to column names:

We can then write the function that will preprocess our samples. The tricky part is to put all the possible pairs of sentences into two lists before passing them to the tokenizer, then un-flatten the result so that each sample has four input IDs, attention masks and so on.

We also define a maximum sequence length (`max_seq_length`) to either pad or truncate our samples such that each sample is the same size. Then, when calling the tokenizer, we use the arguments `truncation=True`, `padding="max_length"` and `max_length=max_seq_length`.

In [ ]:
ending_names = ["ending0", "ending1", "ending2", "ending3"]
max_seq_length = 128

def preprocess_function(examples):
    # Repeat each first sentence four times to go with the four possibilities of second sentences.
    first_sentences = [[context] * 4 for context in examples["sent1"]]
    # Grab all second sentences possible for each context.
    question_headers = examples["sent2"]
    second_sentences = [[f"{header} {examples[end][i]}" for end in ending_names] for i, header in enumerate(question_headers)]
    
    # Flatten everything
    first_sentences = sum(first_sentences, [])
    second_sentences = sum(second_sentences, [])
    
    # Tokenize
    tokenized_examples = tokenizer(
        first_sentences, 
        second_sentences, 
        truncation=True,
        max_length=max_seq_length,
        padding="max_length",
    )
    # Un-flatten
    return {k: [v[i:i+4] for i in range(0, len(v), 4)] for k, v in tokenized_examples.items()}

This function works with one or several samples. In the case of several samples, the tokenizer will return a list of lists of lists for each key: a list of all samples (here 5), then a list of all choices (4) and a list of input IDs:

In [ ]:
examples = datasets["train"][:5]
features = preprocess_function(examples)
print(len(features["input_ids"]), len(features["input_ids"][0]), [len(x) for x in features["input_ids"][0]])

To check we didn't do anything wrong when grouping all possibilities then un-flattening, let's have a look at the decoded inputs for a given sample:

In [ ]:
idx = 3
[tokenizer.decode(features["input_ids"][idx][i]) for i in range(4)]

We can compare it to the ground truth:

In [ ]:
show_one(datasets["train"][3])

This seems right, so we can apply this function to all the samples in our dataset by using the `map` method of the `dataset` object we created earlier. This will apply the function to all the elements of all the splits in `dataset`. This means that our training, validation and testing data will be preprocessed with a single command.

In [ ]:
encoded_datasets = datasets.map(preprocess_function, batched=True)

Even better, the results are automatically cached by the 🤗 Datasets library to avoid spending time on this step the next time you run your notebook. The 🤗 Datasets library is able to detect when the function you pass to `map` has changed (and thus not to use the cached data). For instance, it will properly detect if you change the task in the first cell and rerun the notebook. 🤗 Datasets warns you when it uses cached files and you can pass `load_from_cache_file=False` in the call to `map` to not use the cached files and to force the preprocessing to be applied again.

Note that we passed `batched=True` to encode the text samples together into batches. This is to leverage the full benefit of the fast tokenizer we loaded earlier, which will use multi-threading to treat the text samples in a batch concurrently.

## Fine-tuning the model

Now that our data is ready, we can download the pre-trained model and fine-tune it. Since all our tasks are multiple choice tasks, we use the `AutoModelForMultipleChoice` class. Like with the tokenizer, the `from_pretrained` method will download and cache the model for us. We also define the `IPUConfig` class, which is a class that specifies attributes and configuration parameters to compile and put the model on the device. We initialize it with a config name or a path, which we set earlier.

In [ ]:
from transformers import AutoModelForMultipleChoice
from optimum.graphcore import IPUConfig, IPUTrainer, IPUTrainingArguments

model = AutoModelForMultipleChoice.from_pretrained(model_checkpoint)
ipu_config = IPUConfig.from_pretrained(ipu_config_name, executable_cache_dir=executable_cache_dir)

The warning tells us we are throwing away some weights (the `vocab_transform` and `vocab_layer_norm` layers) and randomly initializing some others (the `pre_classifier` and `classifier` layers). This is normal in this case, because we are removing the head used to pre-train the model on a masked language modelling objective and replacing it with a new head for which we don't have pre-trained weights, so the library warns us we should fine-tune this model before using it for inference, which is exactly what we are going to do.

To instantiate the `IPUTrainer` class, we will need to define:
* `IPUTrainingArguments`, which is a class that contains all the attributes to customize the training.
* How to compute the metrics from the predictions

`IPUTrainingArguments` requires one folder name, which will be used to save the checkpoints of the model, and all other arguments are optional:

In [ ]:
model_name = model_checkpoint.split("/")[-1]
args = IPUTrainingArguments(
    f"{model_name}-finetuned-swag",
    evaluation_strategy = "epoch",
    learning_rate=5e-5,
    per_device_train_batch_size=micro_batch_size,
    per_device_eval_batch_size=micro_batch_size,
    n_ipu=n_ipu,
    gradient_accumulation_steps=gradient_accumulation_steps,
    num_train_epochs=3,
    weight_decay=0.01,
    logging_steps=10,
    dataloader_drop_last=True,
    push_to_hub=False,
    # hub_model_id = f"username-or-organization/{model_name}-finetuned-squad"
)

Here we set the evaluation to be done at the end of each epoch, tweak the learning rate and use the global batch size defined earlier in the notebook through `micro_batch_size`, `gradient_accumulation_steps` and `n_ipu`. We also customize the number of epochs for training, as well as the weight decay.

`push_to_hub` in `IPUTrainingArguments` is necessary if we want to push the model to the [🤗 Models Hub](https://huggingface.co/models) regularly during training. You can remove them if you didn't follow the installation steps at the beginning of this notebook. If you want to save your model locally to a name that is different to the name of the repository it will be pushed to, or if you want to push your model under an organization and not your name space, use the `hub_model_id` argument to set the repo name (it needs to be the full name, including your namespace: for instance `"sgugger/bert-finetuned-swag"` or `"huggingface/bert-finetuned-swag"`).

The last thing to define for before we instantiate `IPUTrainer` is how to compute the metrics from the predictions. We need to define a function for this, which will use `metric`, which we loaded earlier. The only preprocessing we have to do is to calculate `argmax` for our predicted logits:

In [ ]:
import numpy as np

def compute_metrics(eval_predictions):
    predictions, label_ids = eval_predictions
    preds = np.argmax(predictions, axis=1)
    return {"accuracy": (preds == label_ids).astype(np.float32).mean().item()}

Then all we need to do is to pass all of this together with our datasets to `IPUTrainer`:

In [ ]:
from transformers import default_data_collator

trainer = IPUTrainer(
    model,
    ipu_config,
    args,
    train_dataset=encoded_datasets["train"],
    eval_dataset=encoded_datasets["validation"],
    tokenizer=tokenizer,
    data_collator=default_data_collator,
    compute_metrics=compute_metrics,
)

We can now fine-tune our model by calling the `train` method:

In [ ]:
trainer.train()

We can check how our model did with the `evaluate` method:

In [ ]:
trainer.evaluate()

You can now upload the result of the training to the 🤗 Hub:

In [ ]:
# trainer.push_to_hub()

You can also share this model and other users can load it with the identifier "your-username/the-name-you-picked" so for instance:


```python
from transformers import AutoModelForMultipleChoice

model = AutoModelForMultipleChoice.from_pretrained("sgugger/my-awesome-model")
```

## Next steps

Try out the other [IPU-powered Jupyter Notebooks](https://www.graphcore.ai/ipu-jupyter-notebooks) to see how how IPUs perform on other tasks.